# Imports

In [1]:
import pandas as pd
import numpy as np
import heapq

from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from collections import defaultdict
from heapq import heappush, heappop

# Load and prepare the dataset

In [2]:
df = pd.read_csv("roads.csv")

print(df.head())
print(df.shape)
print(df.columns.tolist())

graph = defaultdict(list)

unique_roads = df[['road_id', 'nodes', 'length_km', 'speed_limit']].drop_duplicates()

for _, row in unique_roads.iterrows():
    start, end = row['nodes'].split('-')
    road_info = {
        'road_id' : row['road_id'],
        'nodes': row['nodes'],
        'length_km': row['length_km'],
        'speed_limit': row['speed_limit']
        }
    graph[start].append(road_info)
    graph[end].append(road_info)
df = df[df['avg_speed'] > 0]

print({node: [r['nodes'] for r in roads] for node, roads in graph.items()})

   road_id nodes  length_km  speed_limit  avg_speed  hour
0        1   A-B         52           50         50     0
1        1   A-B         52           50         50     0
2        1   A-B         52           50         48     0
3        1   A-B         52           50         45     0
4        1   A-B         52           50         48     0
(3600, 6)
['road_id', 'nodes', 'length_km', 'speed_limit', 'avg_speed', 'hour']
{'A': ['A-B', 'A-D'], 'B': ['A-B', 'B-C', 'B-E'], 'D': ['A-D', 'D-E'], 'C': ['B-C', 'C-E', 'C-F'], 'E': ['B-E', 'C-E', 'D-E', 'E-F', 'E-G'], 'F': ['C-F', 'E-F', 'F-G'], 'G': ['E-G', 'F-G']}


# RF Features

In [3]:
df_avg = df[df['avg_speed'] > 0].copy()

df_avg['hour_sin'] = np.sin(2 * np.pi * df_avg['hour'] / 24)
df_avg['hour_cos'] = np.cos(2 * np.pi * df_avg['hour'] / 24)

road_ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
road_encoded = road_ohe.fit_transform(df_avg[['road_id']])

X = np.concatenate([
        df_avg[['length_km', 'speed_limit', 'hour_sin', 'hour_cos']].values,
        road_encoded
        ], axis = 1)


y = df_avg['avg_speed'].values

print("Feature matrix shape:", X.shape)
print("Target vector shape:", y.shape)

Feature matrix shape: (3600, 14)
Target vector shape: (3600,)


# Training the model

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

rf_speed = RandomForestRegressor(
    n_estimators=300,
    max_depth = 20,
    random_state = 42,
    n_jobs=-1
)

rf_speed.fit(X_train, y_train)

y_pred = rf_speed.predict(X_test)


print("RF RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("RF R2 score:", r2_score(y_test, y_pred))

RF RMSE: 2.975668586727318
RF R2 score: 0.9196770063078308


# Predict speed function

In [5]:
def predict_speed(road_id, hour):
    hour_sin = np.sin(2 * np.pi * hour / 24)
    hour_cos = np.cos(2 * np.pi * hour / 24)

    road_row = df_avg[df_avg['road_id'] == road_id].iloc[0]

    road_df = pd.DataFrame({'road_id': [road_id]})
    road_vec = road_ohe.transform(pd.DataFrame({'road_id':[road_id]}))
    
    features = np.concatenate([
        df_avg[df_avg['road_id'] == road_id][['length_km', 'speed_limit']].values[0].reshape(1,-1),
        np.array([[hour_sin, hour_cos]]),
        road_vec
    ], axis = 1)

    return rf_speed.predict(features)[0]

# Heuristic function

In [6]:
def heuristic_time(node, goal_distances_km):
    max_speed = df_avg['speed_limit'].max()
    return goal_distances_km[node] / max_speed

# A* search

In [7]:
def a_star_time(graph, start, goal, start_time, goal_distances_km):
    open_set = []
    heappush(open_set, (0, start_time, start, []))

    visited = set()

    while open_set:
        f_score, current_time, node, path = heappop(open_set)

        if node == goal:
            return path, current_time

        if (node, current_time.hour) in visited:
            continue
        visited.add((node, current_time.hour))

        for road in graph[node]:
            if path and road['road_id'] == path[-1]['road_id']:
                continue

            other = (
                road['nodes'].replace(f"{node}-", "")
                if road['nodes'].startswith(node)
                else road['nodes'].replace(f"-{node}", "")
            )

            hour = current_time.hour
            speed = predict_speed(road['road_id'], hour)

            travel_time_hours = road['length_km'] / speed
            travel_time = timedelta(hours=travel_time_hours)
            arrival_time = current_time + travel_time

            g_score = (arrival_time - start_time).total_seconds() / 3600

            h_score = heuristic_time(other, goal_distances_km)

            f_score_next = g_score + h_score

            heappush(
                open_set,
                (f_score_next, arrival_time, other, path + [road])
            )

    return None, None


# Goal distance function

In [8]:
def compute_goal_distances(graph, goal):
    dist = {node: float('inf') for node in graph}
    dist[goal] = 0
    pq = [(0, goal)]

    while pq:
        current_dist, node = heapq.heappop(pq)

        if current_dist > dist[node]:
            continue

        for road in graph[node]:
            other = (
                road['nodes'].replace(f"{node}-", "")
                if road['nodes'].startswith(node)
                else road['nodes'].replace(f"-{node}", "")
            )

            new_dist = current_dist + road['length_km']
            if new_dist < dist[other]:
                dist[other] = new_dist
                heapq.heappush(pq, (new_dist, other))

    return dist

# Interactive layout

In [10]:
def run_path_planner():
    print("Available nodes:", list(graph.keys()))
    start = input("Enter starting node: ").strip()
    while start not in graph:
        start = input("Invalid node. Enter starting node: ").strip()

    goal = input("Enter goal node: ").strip()
    while goal not in graph:
        goal = input("Invalid node. Enter goal node: ").strip()

    now = datetime.now()
    goal_distances_km = compute_goal_distances(graph, goal)
    path, arrival_time = a_star_time(graph, start, goal, now, goal_distances_km)

    if path is None:
        print("No path found!")
        return

    print(f"\nShortest path from {start} to {goal}:")
    current_time = now
    total_time = timedelta()
    nodes_path = [start]
    for road in path:
        other = road['nodes'].replace(f"{nodes_path[-1]}-", "") if road['nodes'].startswith(nodes_path[-1]) else road['nodes'].replace(f"-{nodes_path[-1]}", "")
        speed = predict_speed(road['road_id'], current_time.hour)
        travel_time_hours = road['length_km'] / speed
        travel_time = timedelta(hours=travel_time_hours)
        arrival_time_node = current_time + travel_time
        print(f"{nodes_path[-1]} -> {other} via road {road['road_id']} | "
              f"Length: {road['length_km']} km, Predicted speed: {speed:.2f} km/h, "
              f"Time: {travel_time}")
        print(current_time)
        total_time += travel_time
        current_time = arrival_time_node
        nodes_path.append(other)

    total_seconds = int(total_time.total_seconds())
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60

    print(f"\nEstimated total travel time: {hours}h {minutes}m")
    print(f"\nEstimated Time of Arrival at {goal}: {current_time.strftime('%H:%M')}")
    print("Node path:", " -> ".join(nodes_path))

run_path_planner()

Available nodes: ['A', 'B', 'D', 'C', 'E', 'F', 'G']


Enter starting node:  G
Enter goal node:  A



Shortest path from G to A:
G -> E via road 9 | Length: 87 km, Predicted speed: 50.95 km/h, Time: 1:42:27.237145
2026-01-13 22:03:09.874527
E -> B via road 4 | Length: 105 km, Predicted speed: 67.68 km/h, Time: 1:33:05.486942
2026-01-13 23:45:37.111672
B -> A via road 1 | Length: 52 km, Predicted speed: 47.67 km/h, Time: 1:05:26.929858
2026-01-14 01:18:42.598614

Estimated total travel time: 4h 20m

Estimated Time of Arrival at A: 02:24
Node path: G -> E -> B -> A
